In [2]:
from langchain.llms import Ollama
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import CSVLoader, PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain.chains import LLMChain, RetrievalQA

In [3]:
llm = Ollama(model="llama2")

In [4]:
embeddings = OllamaEmbeddings(model="llama2")

In [5]:
dictionary = "../data/Data Dictionary.pdf"
data = "../data/data.csv"

In [9]:
dict_loader = PyPDFLoader(dictionary)
data_loader = CSVLoader(data)

In [10]:
data_docs = data_loader.load()

In [11]:
dict_doc = dict_loader.load()

In [24]:
print(len(dict_doc), dict_doc[0])

1 page_content='Data\nDictionary\nColumn\nDescription\nclnt_id\nClient\nID\nbank_id\nBank\nID\nacc_id\nAccount\nID\ntxn_id\nTransaction\nID\ntxn_date\nTransaction\ndate\ndesc\nDescription\namt\nAmount\ncat\nCategory\nof\nthe\ntransaction\nmerchant\nMerchant\nof\nthe\ntransaction' metadata={'source': '../data/Data Dictionary.pdf', 'page': 0}


In [13]:
print(data_docs[1])

page_content='clnt_id: 6\nbank_id: 1\nacc_id: 1\ntxn_id: 27\ntxn_date: 31/07/2023 0:00\ndesc: CLOC Advance\namt: 6.286\ncat: Shops\nmerchant: NA' metadata={'source': '../data/data.csv', 'row': 1}


In [ ]:
# dict_chroma_db = Chroma.from_documents(dict_doc, embeddings, persist_directory="../dict_chroma_db")

In [16]:
dict_chroma_db = Chroma(persist_directory="../dict_chroma_db", embedding_function=embeddings)

In [ ]:
# data_chroma_db = Chroma.from_documents(data_docs[:100], embeddings, persist_directory="../100_chunk_chroma_db")

In [17]:
data_chroma_db = Chroma(persist_directory="../100_chunk_chroma_db", embedding_function=embeddings)

In [57]:
len(data_chroma_db)

100

In [97]:
data_chroma_db.get()['documents']

['clnt_id: 67\nbank_id: 1\nacc_id: 1\ntxn_id: 27\ntxn_date: 19/08/2023 0:00\ndesc: Apple Cash\namt: 10.836\ncat: Shops\nmerchant: Apple',
 'clnt_id: 51\nbank_id: 1\nacc_id: 1\ntxn_id: 8\ntxn_date: 19/07/2023 0:00\ndesc: 07-18-23 VISA DIRECT CA 2990 APPLE CASH INST XFER VISA MONEY TRANSFER CREDIT\namt: 34.476\ncat: Shops\nmerchant: NA',
 'clnt_id: 110\nbank_id: 1\nacc_id: 1\ntxn_id: 4\ntxn_date: 12/06/2023 0:00\ndesc: APPLE CASH VISA DIRECT CA 25\namt: 9.456\ncat: Shops\nmerchant: Apple',
 'clnt_id: 87\nbank_id: 1\nacc_id: 1\ntxn_id: 126\ntxn_date: 08/09/2023 0:00\ndesc: APPLE CASH INST XFER VISA DIRECT CA\namt: 3.546\ncat: Shops\nmerchant: Apple',
 'clnt_id: 87\nbank_id: 1\nacc_id: 1\ntxn_id: 57\ntxn_date: 31/08/2023 0:00\ndesc: APPLE CASH INST XFER 240 CA\namt: 45.31\ncat: Shops\nmerchant: Apple',
 'clnt_id: 104\nbank_id: 1\nacc_id: 1\ntxn_id: 105\ntxn_date: 02/06/2023 0:00\ndesc: BEST BUY 1200 FLINT MI\namt: 4.666\ncat: Shops\nmerchant: Best Buy',
 'clnt_id: 73\nbank_id: 1\nacc_id: 1

In [95]:
dict_chroma_db.get()

{'ids': ['d5859313-fef4-4da4-be12-59537cb1a52f'],
 'embeddings': None,
 'metadatas': [{'page': 0, 'source': '../data/Data Dictionary.pdf'}],
 'documents': ['Data\nDictionary\nColumn\nDescription\nclnt_id\nClient\nID\nbank_id\nBank\nID\nacc_id\nAccount\nID\ntxn_id\nTransaction\nID\ntxn_date\nTransaction\ndate\ndesc\nDescription\namt\nAmount\ncat\nCategory\nof\nthe\ntransaction\nmerchant\nMerchant\nof\nthe\ntransaction'],
 'uris': None,
 'data': None}

In [12]:
data_chroma_db.persist()

In [13]:
dict_chroma_db.persist()

In [21]:
dict_retriever = dict_chroma_db.as_retriever()

In [22]:
data_retriever = data_chroma_db.as_retriever()

In [130]:
print(data_retriever.schema_json())

ValueError: Value not declarable with JSON Schema, field: name='vectorstore' type=VectorStore required=True

In [20]:
from langchain.agents import Tool
from pydantic.v1 import BaseModel, Field
from langchain.prompts import PromptTemplate


In [107]:
prompt_template = PromptTemplate(
        input_variables=["context", "client_id"],
        template="""
        Given these transactions: {context}
        
        , please answer the question: {question}
        for a client with clnt_id: {client_id}
        
        Make sure you answer in a human readable sentence.
        Answer directly, don't refer to the how you answered the question.
        
        This is what each column/keyword means:
        Column ->> Description
        clnt_id ->> Client ID
        bank_id ->>  Bank ID
        acc_id ->>  Account ID
        txn_id ->> Transaction ID
        txn_date ->> Transaction date
        desc ->> Description
        amt ->> Amount
        cat ->> Category of the transaction
        merchant ->> Merchant of the transaction
        """,
    )

In [108]:
class DocumentInput(BaseModel):
    question: str = Field()

In [109]:
dict_tool = Tool(
    args_schema = DocumentInput,
    name="Data Dictionary",
    description="useful for when you need to know the description of colums in the csv file.",
    func=RetrievalQA.from_chain_type(llm=llm, retriever=dict_retriever)
)

In [110]:
data_tool = Tool(
    args_schema = DocumentInput,
    name="Data",
    description="useful for when you need to answer questions about customer transactions. This is a csv file.",
    func=RetrievalQA.from_chain_type(llm=llm, retriever=data_retriever, chain_type_kwargs={"prompt": prompt_template},),
)

In [111]:
tools = [
    # dict_tool, 
    data_tool,
]

In [112]:
from langchain.agents import AgentType, initialize_agent

In [113]:
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True, 
)

In [114]:
from langchain.globals import set_debug

set_debug(True)

In [116]:
agent({"input":"How much did I spend on 01/08/2023?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How much did I spend on 01/08/2023?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "How much did I spend on 01/08/2023?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools:\n\nData: useful for when you need to answer questions about customer transactions. This is a csv file.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Data]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThough

KeyboardInterrupt: 

In [106]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=data_retriever, chain_type="stuff", chain_type_kwargs={"prompt": prompt_template,})

In [ ]:
qa_chain({
    'query':'How much did I spend on 01/08/2023. My client Id is 6.',
})

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "How much did I spend on 01/08/2023. My client Id is 6.",
  "client_id": "6"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/error] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] s] Chain run errored with error:
"ValueError(\"Missing some input keys: {'client_id'}\")Traceback (most recent call last):\n\n\n  File \"c:\\Users\\USER\\Desktop\\moneylion\\env\\Lib\\site-packages\\langchain\\chains\\base.py\", line 151, in invoke\n    self._validate_inputs(inputs)\n\n\n  File \"c:\\Users\\USER\\Desktop\\moneylion\\env\\Lib\\site-packages\\langchain\\chains\\base.py\", line 279, in _validate_inputs\n    raise ValueError(f\"Missing some input keys: {missing_keys}\")\n\n\nValueError: Missing some input keys: {'client_id'}"
[chain/error] [1:chain:RetrievalQA] [7.81s] Chain run errored with error:
"ValueError(\"Missing some input keys: {'client_id

ValueError: Missing some input keys: {'client_id'}

In [117]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [118]:
template="""
        Given these transactions: {context}
        
        for a client with clnt_id: {client_id}
        ,please answer the question: {question}
        
        Make sure you answer in a human readable sentence.
        Answer directly, don't refer to the how you answered the question.
        
        This is what each column/keyword means:
        Column ->> Description
        clnt_id ->> Client ID
        bank_id ->>  Bank ID
        acc_id ->>  Account ID
        txn_id ->> Transaction ID
        txn_date ->> Transaction date
        desc ->> Description
        amt ->> Amount
        cat ->> Category of the transaction
        merchant ->> Merchant of the transaction
        """
        
prompt = ChatPromptTemplate.from_template(template)

In [125]:
from langchain.prompts import (
    ChatPromptTemplate, HumanMessagePromptTemplate, 
    MessagesPlaceholder
)

from langchain_core.messages import SystemMessage

human_message_template = HumanMessagePromptTemplate.from_template(template)

chat_prompt = ChatPromptTemplate.from_messages(
    [   SystemMessage(
            content=(
                "You are a helpful assistant that serves as customers accountant"
                "You asnwer their questions based on data given to you."
                "If you don't have their answer, reply that you cannot help in the moment."
            )
        ),
        MessagesPlaceholder(variable_name="desc"), human_message_template]
)

In [127]:
from langchain_core.messages import HumanMessage
clnt_id = 6

human_message = HumanMessage(
    f"My client Id is {clnt_id}"
)

chat_prompt.format_prompt(
    desc = [human_message], client_id=clnt_id,
    context=data_retriever,
    question="How much did I spend on 01/08/2023. My client Id is 6."
).to_messages()

[HumanMessage(content='My client Id is 6'),
 HumanMessage(content="\n        Given these transactions: tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000028C780B7910>\n        \n        for a client with clnt_id: 6\n        ,please answer the question: How much did I spend on 01/08/2023. My client Id is 6.\n        \n        Make sure you answer in a human readable sentence.\n        Answer directly, don't refer to the how you answered the question.\n        \n        This is what each column/keyword means:\n        Column ->> Description\n        clnt_id ->> Client ID\n        bank_id ->>  Bank ID\n        acc_id ->>  Account ID\n        txn_id ->> Transaction ID\n        txn_date ->> Transaction date\n        desc ->> Description\n        amt ->> Amount\n        cat ->> Category of the transaction\n        merchant ->> Merchant of the transaction\n        ")]

In [ ]:
llm.invoke()

In [175]:
template="""
        You are a helpful assistant that serves as customer care service for our company.
        You asnwer their questions.
        If you don't have their answer, reply that you cannot help in the moment.
                
        This is our custmers' data: {context}
        
        please answer this customer's question directly to them: {question}
        Their client clnt_id: {client_id}
        
        
        Make sure you answer in one human readable sentence.
        Answer directly, like you are in charge.
        Don't add extra informations or extra questions, or refer to your sources and don't let the customers know about the documents.
        
        
        This is what each column/keyword means:
        Column ->> Description
        clnt_id ->> Client ID
        bank_id ->>  Bank ID
        acc_id ->>  Account ID
        txn_id ->> Transaction ID
        txn_date ->> Transaction date
        desc ->> Description
        amt ->> Amount
        cat ->> Category of the transaction
        merchant ->> Merchant of the transaction
        """
        
prompt = ChatPromptTemplate.from_template(template)

In [176]:
from operator import itemgetter

def invoke_chain(question: str, client_id=6):
    chain = (
        {
            'context': itemgetter("question") | data_retriever, "question": itemgetter("question"), "client_id": itemgetter('client_id'), 
         }
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return chain.invoke({
        "client_id": client_id,
        "question": question
    })

In [180]:
reply = invoke_chain(question="How much did I spend on August 1st, 2023?", client_id=86)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "client_id": 86,
  "question": "How much did I spend on August 1st, 2023?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,client_id>] Entering Chain run with input:
{
  "client_id": 86,
  "question": "How much did I spend on August 1st, 2023?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,client_id> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "client_id": 86,
  "question": "How much did I spend on August 1st, 2023?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,client_id> > 3:chain:RunnableSequence > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "client_id": 86,
  "question": "How much did I spend on August 1st, 2023?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,client_id> > 3:chain:RunnableSequence > 4:chain:RunnableLambda

In [179]:
print(reply)

Dear valued customer,

According to our records, your transaction on August 1st, 2023, had a total spend of $2.252.

Please let me know if you have any further questions or concerns.
